In [1]:
# Most Basic
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import math
from os import path
import sys
from datetime import datetime
from scipy.stats import norm
import scipy.stats as ss
import copy
import warnings
warnings.simplefilter('error')
import datetime

# Filter Specific Functionalities
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints

# Gradient Based
import autograd.numpy as a_np
import autograd.scipy.stats as a_ss
from autograd import grad, multigrad_dict, elementwise_grad, jacobian

# Self Defined functions5
from DataMaker_New import dataCleaner_new
from ESN import EchoState
from Loss_diag_updateV import loss

In [2]:
columns = ["date","exdate","strike_price","best_bid","best_offer","volume","impl_volatility"]
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 12, 31)
train_valid_split = 0.936
valid_test_split = 0.94
u_dim = 10 
topK = 5
cleaner = dataCleaner_new(columns, u_dim, topK, start, end, train_valid_split, valid_test_split)
return_dict, price_dict = cleaner.make_return_price()
risk_dict = cleaner.make_risk()
train_data, valid_data, test_data = cleaner.make_final_data(price_dict, return_dict, risk_dict)
num_train_day, num_train_obs = cleaner.get_train_num()
num_valid_day, num_valid_obs = cleaner.get_valid_num()
num_test_day, num_test_obs = cleaner.get_test_num()

# train_data[np.datetime64('2019-01-02T00:00:00.000000000')]

Attached 351 days of train data
Attached 1755 train option observations
Attached 1 days of validation data
Attached 5 validation option observations
Attached 23 days of test data
Attached 115 test option observations


In [3]:
# Define transition Dynamics
# Positional requirement, no real effect, will be replaced by ESN and BS defined further down
def fx(x, dt):
    return x*dt
def hx(x):
    return x

def sigmoid(x):
    return np.divide(1, 1+np.exp(-x))


def ESNf(theta_now, dt, **fx_args):
    """
    :params theta_now theta_t
    
    **fx_args
    :params G reservoir transition matrix
    :params G_in input translation matrix
    :params b reservoir bias vector
    :params u_quad_now u_t^2
    """
    G = fx_args["G"]
    G_in = fx_args["G_in"]
    b = fx_args["b"]
    u_quad_now = fx_args["u_quad_now"]
    theta_next = sigmoid(np.matmul(G, theta_now) + np.matmul(G_in, u_quad_now) + b)
    return theta_next
    

def BSf(theta_t, **hx_args):
    """
    :params theta_t current state vector
    
    **hx_args
    :params p_t current asset price
    :params r_t current asset price
    :params K_t strike price, 1d vector
    :params T_t maturity time, 1d vector
    """
    p_t = hx_args["p_t"]
    r_t = hx_args["r_t"]
    K_t = hx_args["K_t"]
    T_t = hx_args["T_t"]
    
    volatility_std = np.average(theta_t)
    volatility_var = np.power(volatility_std, 2)
    
    dividor = np.sqrt(volatility_var * T_t)
    d_pls = (np.log(p_t/K_t) + np.multiply(r_t+volatility_var/2, T_t)) / dividor
    d_mns = (np.log(p_t/K_t) + np.multiply(r_t-volatility_var/2, T_t)) / dividor
    y_t = np.multiply(p_t, norm.cdf(d_pls)) - np.multiply(np.multiply(K_t, np.exp(-r_t*T_t)), norm.cdf(d_mns))
    return y_t

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)


# 睡觉专区

In [6]:
np.random.seed(1234)
gradient = grad(loss)


forward_k = 20

error_for_all_theta_dim = []
best_itr_for_all_theta_dim = []
best_train_itr_for_all_theta_dim = []
params_for_all_theta_dim = []
ms_s_for_all_theta_dim = []
Cs_s_for_all_theta_dim = []

train_pred_ms_for_all_theta_dim = []
pred_ms_for_all_theta_dim = []


for theta_dim in np.arange(8, 9, 2):
    step = 0.02
    m_t_1 = [np.array([0]), np.array([0]), np.array([0]), np.array([0]), np.array([0])]
    v_t_1 = [np.array([0]), np.array([0]), np.array([0]), np.array([0]), np.array([0])]
    beta_1 = 0.9
    beta_2 = 0.999
    epsillon = 1e-8
    
    print("theta_dim = {}".format(theta_dim))
    alpha = 0.05
    connectivity = 0.4
    spectral_radius = 0.98
    bday_p_year = 252
    bias = -2.0

    v_scale = 4.0
    target_var = 1e-6
    chi_mu = math.sqrt(2) * math.gamma((theta_dim+1)/2)/math.gamma(theta_dim/2) * 0.0001
    w = target_var+chi_mu

    # initialize ESN dynamics
    ESN = EchoState(theta_dim, u_dim, connectivity, spectral_radius, bias)
    v = v_scale

    last_avg_error = 100
    best_itr = 0
    best_train_error = 100
    best_train_itr = 0
    
    #%%%%%% Error trajectories
    
    test_error_traj = []
    params_traj = []
    ms_s_traj = []
    Cs_s_traj = []
    
    train_pred_ms_traj = []
    pred_ms_traj = []
    
    for em_itr in range(6):
        dt=1
        points = MerweScaledSigmaPoints(theta_dim, alpha=.001, beta=2., kappa=0)
        ukf = UnscentedKalmanFilter(dim_x=ESN.theta_dim, dim_z=topK, dt=1, fx=fx, hx=hx, points=points)
        ukf.x = np.ones(ESN.theta_dim)* 0.006 # initial state
        ukf.P *= 0.01 # initial uncertainty
        ukf.R = np.diag([v]*topK) # 1 standard
        ukf.Q = np.eye(theta_dim)*w

        ms = [ukf.x.copy()]
        Cs = [ukf.P.copy()]
        u_quad_s = []
        ps = []
        rs = []
        ys = []
        Ks = []
        us = []
        Ts = []

        all_days = sorted(list(train_data.keys()))
        for day in all_days:
            # get observable data
            ## Stock Wise and market wise data
            r_t = train_data[day][1] 
            u_t = train_data[day][2].reshape([-1]) * math.sqrt(bday_p_year)
            p_t = train_data[day][3]

            ## Option wise data
            y_t = train_data[day][0]["spotclose"].values
            K_t = train_data[day][0]["strike_price"].values
            today = [d.date() for d in train_data[day][0]["date"]]
            expireday = [d.date() for d in train_data[day][0]["exdate"]]
            T_t = np.busday_count(today, expireday) / bday_p_year


            ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
            ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
            ms.append(ukf.x_post.copy())
            Cs.append(ukf.P_post.copy())
            u_quad_s.append(np.power(u_t,2))
            us.append(u_t)
            ps.append(p_t)
            rs.append(r_t)
            ys.append(y_t)
            Ks.append(K_t)
            Ts.append(T_t)

        ms = np.array(ms)
        Cs = np.array(Cs)
        ms_s, Cs_s, _, cvs_s = ukf.rts_smoother(ms, Cs, fx=ESNf, G=ESN.G, G_in=ESN.G_in, b=ESN.b, u_s=us)


        params = [copy.copy(ESN.G), copy.copy(ESN.G_in), copy.copy(ESN.b), copy.copy(w), copy.copy(v)]

        
        try:
            curr_loss = loss(params,
                             ms_s, Cs_s, cvs_s,
                             u_quad_s,
                             ps, rs, Ks, Ts, ys,
                             num_train_obs,
                             reg=alpha)
            curr_grad = gradient(params,
                                 ms_s, Cs_s, cvs_s,
                                 u_quad_s,
                                 ps, rs, Ks, Ts, ys,
                                 num_train_obs,
                                 reg=alpha)

######################## Adam Optimizer ###########################
            
            m_t = [None, None, None, None, None]
            v_t = [None, None, None, None, None]
            m_t_hat = [None, None, None, None, None]
            v_t_hat = [None, None, None, None, None]

            m_t[0] = beta_1 * m_t_1[0] + (1-beta_1) * curr_grad[0]
            v_t[0] = beta_2 * v_t_1[0] + (1-beta_2) * (curr_grad[0] * curr_grad[0])
            m_t_hat[0] = m_t[0] / (1-beta_1 ** (em_itr+1))
            v_t_hat[0] = v_t[0] / (1-beta_2 ** (em_itr+1))
            params[0] -= step * m_t_hat[0] / (np.sqrt(v_t_hat[0]) + epsillon)

            m_t[1] = beta_1 * m_t_1[1] + (1-beta_1) * curr_grad[1]
            v_t[1] = beta_2 * v_t_1[1] + (1-beta_2) * (curr_grad[1] * curr_grad[1])
            m_t_hat[1] = m_t[1] / (1-beta_1 ** (em_itr+1))
            v_t_hat[1] = v_t[1] / (1-beta_2 ** (em_itr+1))
            params[1] -= step * m_t_hat[1] / (np.sqrt(v_t_hat[1]) + epsillon)

            m_t[2] = beta_1 * m_t_1[2] + (1-beta_1) * curr_grad[2]
            v_t[2] = beta_2 * v_t_1[2] + (1-beta_2) * (curr_grad[2] * curr_grad[2])
            m_t_hat[2] = m_t[2] / (1-beta_1 ** (em_itr+1))
            v_t_hat[2] = v_t[2] / (1-beta_2 ** (em_itr+1))
            params[2] -= step * m_t_hat[2] / (np.sqrt(v_t_hat[2]) + epsillon)
            
            m_t[3] = beta_1 * m_t_1[3] + (1-beta_1) * curr_grad[3]
            v_t[3] = beta_2 * v_t_1[3] + (1-beta_2) * (curr_grad[3] * curr_grad[3])
            m_t_hat[3] = m_t[3] / (1-beta_1 ** (em_itr+1))
            v_t_hat[3] = v_t[3] / (1-beta_2 ** (em_itr+1))
            params[3] -= step * m_t_hat[3] / (np.sqrt(v_t_hat[3]) + epsillon)

            m_t[4] = beta_1 * m_t_1[4] + (1-beta_1) * curr_grad[4]
            v_t[4] = beta_2 * v_t_1[4] + (1-beta_2) * (curr_grad[4] * curr_grad[4])
            m_t_hat[4] = m_t[4] / (1-beta_1 ** (em_itr+1))
            v_t_hat[4] = v_t[4] / (1-beta_2 ** (em_itr+1))
            params[4] -= step * m_t_hat[4] / (np.sqrt(v_t_hat[4]) + epsillon)

            m_t = m_t_1
            v_t = v_t_1

###################################################################

        except RuntimeWarning or LinAlgError or OverflowError or w <= 0:
            print("set w to 1e-6 ensure positive")
            pass

        ESN.G, ESN.G_in, ESN.b, _, __ = params
        w = max(0.000001, params[3][0]) # Change form arraybox back to scalar
        v = max(0.000001, params[4][0]) # Change form arraybox back to scalar

        
################################################################### Training Performance
        
        train_pred_ms = []
        train_errors = []
        np.delete(ms_s,0,0)
        np.delete(Cs_s,0,0)
        for i in range(0, len(ms_s)-1):

            ukf.predict(fx = ESNf,  G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=u_quad_s[i])
            train_pred_ms.append(np.average(ukf.x_prior))
            pred_price = BSf(ukf.x_prior, p_t=ps[i], r_t=rs[i], K_t=Ks[i], T_t=Ts[i])
            error = np.average(np.abs(ys[i]-pred_price) /ys[i])
            train_errors.append(error)
            ukf.update(ys[i], hx=BSf, p_t=ps[i], r_t=rs[i], K_t=Ks[i], T_t=Ts[i])
        
        train_pred_ms_traj.append(train_pred_ms)
        avg_train_error = np.average(train_errors)
################################################################### Validation Performance
        ukf.x = ms_s[-1]
        ukf.P = Cs_s[-1]
        valid_errors = []
        pred_ms = []
        
        all_valid_days = sorted(list(valid_data.keys()))
        for day in all_valid_days:
            # get observable data
            ## Stock Wise and market wise data
            r_t = valid_data[day][1]  
            u_t = valid_data[day][2].reshape([-1]) * math.sqrt(bday_p_year)
            p_t = valid_data[day][3]

            ## Option wise data
            y_t = valid_data[day][0]["spotclose"].values
            K_t = valid_data[day][0]["strike_price"].values
            today = [d.date() for d in valid_data[day][0]["date"]]
            expireday = [d.date() for d in valid_data[day][0]["exdate"]]
            T_t = np.busday_count(today, expireday) / bday_p_year

            ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
            pred_ms.append(ukf.x_prior)
            pred_price = BSf(ukf.x_prior, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
            error = np.average(np.abs(y_t-pred_price) /y_t)
            valid_errors.append(error)
            
            ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
            
        avg_valid_error = np.average(valid_errors) 

################################################################### Testing Performance
        
        # forward K step prediction
        k_step_test_error = []
        
        for k in range(1, min(forward_k+1, num_test_day+1)):
    
            test_errors = []
            
            
            all_test_days = sorted(list(test_data.keys()))
            
            # For loop for base shift, shift of last updated ukf timestep
            for idx in range(len(all_test_days) - k):
                
                ukf_pred = copy.deepcopy(ukf)
                
                # For loop for k predictions forward
                for shift in range(k) :
                    # get observable data
                    ## Stock Wise and market wise data
                    r_t = test_data[all_test_days[idx+shift]][1] 
                    u_t = test_data[all_test_days[idx+shift]][2].reshape([-1]) * math.sqrt(bday_p_year)
                    p_t = test_data[all_test_days[idx+shift]][3]

                    ## Option wise data
                    y_t = test_data[all_test_days[idx+shift]][0]["spotclose"].values
                    K_t = test_data[all_test_days[idx+shift]][0]["strike_price"].values
                    today = [d.date() for d in test_data[all_test_days[idx+shift]][0]["date"]]
                    expireday = [d.date() for d in test_data[all_test_days[idx+shift]][0]["exdate"]]
                    T_t = np.busday_count(today, expireday) / bday_p_year

                    ukf_pred.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
                    
                    # append 1 step prediction
                    if k == 1:
                        pred_ms.append(ukf_pred.x_prior)
                
                
                pred_price = BSf(ukf_pred.x_prior, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
                error = np.average(np.abs(y_t-pred_price) /y_t)
                test_errors.append(error)
                
                
                # shift base one step forward
                r_t = test_data[all_test_days[idx]][1] 
                u_t = test_data[all_test_days[idx]][2].reshape([-1]) * math.sqrt(bday_p_year)
                p_t = test_data[all_test_days[idx]][3]

                ## Option wise data
                y_t = test_data[all_test_days[idx]][0]["spotclose"].values
                K_t = test_data[all_test_days[idx]][0]["strike_price"].values
                today = [d.date() for d in test_data[all_test_days[idx]][0]["date"]]
                expireday = [d.date() for d in test_data[all_test_days[idx]][0]["exdate"]]
                T_t = np.busday_count(today, expireday) / bday_p_year

                ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
                ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
            
            if test_errors != []:
                avg_test_error = np.average(test_errors)
                k_step_test_error.append(avg_test_error)
            else:
                pass
            
            
################################################################### Show details
        print("EM Iter {} | Loss {:.4f} | Train Error {:.4f} | Val Error {:.4f} | Test 1-Error {:.4f} | Test max-Error {:.4f} |"  
              " Dyn var {:.2f} | Obs var {:.2f}".format(em_itr, curr_loss,
                                                        avg_train_error, avg_valid_error, k_step_test_error[0], k_step_test_error[-1],
                                                        w, v))
        
        if last_avg_error < avg_valid_error and em_itr-best_itr >= 10:
            break
        elif last_avg_error < avg_valid_error and em_itr-best_itr < 10:
            pass
        elif last_avg_error > avg_valid_error and em_itr-best_itr < 10:
            last_avg_error = avg_valid_error
            model_performance = avg_test_error
            best_itr = em_itr
            
        if best_train_error > avg_train_error and em_itr-best_train_itr < 10:
            best_train_error = avg_train_error
            best_train_itr = em_itr
        elif best_train_error < avg_train_error:
            step = 0.02
    
        test_error_traj.append(k_step_test_error)
        params_traj.append(copy.deepcopy(params))
        ms_s_traj.append(ms_s)
        Cs_s_traj.append(Cs_s)
        pred_ms_traj.append(pred_ms)
    
    error_for_all_theta_dim.append(test_error_traj)
    best_itr_for_all_theta_dim.append(best_itr)
    best_train_itr_for_all_theta_dim.append(best_train_itr)
    params_for_all_theta_dim.append(params_traj)
    ms_s_for_all_theta_dim.append(ms_s_traj)
    print("just appended once")
    Cs_s_for_all_theta_dim.append(Cs_s_traj)
    pred_ms_for_all_theta_dim.append(pred_ms_traj)
    train_pred_ms_for_all_theta_dim.append(train_pred_ms_traj)
    
    print("best val error: {} at {}th iteration, Model Performance: {} | theta_dim = {}".format(last_avg_error,
                                                                                                best_itr,
                                                                                                model_performance,
                                                                                                theta_dim))
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

theta_dim = 8
EM Iter 0 | Loss 1872.5563 | Train Error 0.5098 | Val Error 0.5706 | Test 1-Error 0.3046 | Test max-Error 0.1838 | Dyn var 0.02 | Obs var 4.02
EM Iter 1 | Loss 35.8775 | Train Error 0.5276 | Val Error 0.5412 | Test 1-Error 0.2665 | Test max-Error 0.1459 | Dyn var 0.04 | Obs var 4.03
EM Iter 2 | Loss 45.3103 | Train Error 0.5504 | Val Error 0.5182 | Test 1-Error 0.2415 | Test max-Error 0.1207 | Dyn var 0.05 | Obs var 4.05
EM Iter 3 | Loss 60.0060 | Train Error 0.5739 | Val Error 0.4973 | Test 1-Error 0.2249 | Test max-Error 0.1081 | Dyn var 0.04 | Obs var 4.06
EM Iter 4 | Loss 45.9304 | Train Error 0.5957 | Val Error 0.4823 | Test 1-Error 0.2177 | Test max-Error 0.1119 | Dyn var 0.05 | Obs var 4.07
EM Iter 5 | Loss 58.5871 | Train Error 0.6208 | Val Error 0.4608 | Test 1-Error 0.2111 | Test max-Error 0.1300 | Dyn var 0.04 | Obs var 4.08
just appended once
best val error: 0.460752308083041 at 5th iteration, Model Performance: 0.13001278406862415 | theta_dim = 8
++++++++++++

# Plot Prediction error vs forward step & Volatility trajectory  

In [7]:
if len(error_for_all_theta_dim) == 0:
    error_for_all_theta_dim.append(test_error_traj)
    best_itr_for_all_theta_dim.append(best_itr)
    best_train_itr_for_all_theta_dim.append(best_train_itr)
    params_for_all_theta_dim.append(params_traj)
    ms_s_for_all_theta_dim.append(ms_s_traj)
    Cs_s_for_all_theta_dim.append(Cs_s_traj)
    pred_ms_for_all_theta_dim.append(pred_ms_traj)
    train_pred_ms_for_all_theta_dim.append(train_pred_ms_traj)
    print("just appended once")
    
warnings.simplefilter('error')


all_pred_days = sorted(list(valid_data.keys()) + list(test_data.keys()))
for k in range(len(ms_s_for_all_theta_dim)):

    sns.set()
    fig = plt.figure(figsize=(23, 6))
    spec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[1, 2], height_ratios=[1])

    
    mean_iteration = 3 #int((best_train_itr_for_all_theta_dim[k] + best_itr_for_all_theta_dim[k])/2) ###########
    iterations = range(mean_iteration, mean_iteration+1) ############
    
    title = "THETA={}_ALPHA={}_ITR={}".format(k+theta_dim, alpha, mean_iteration)
    fig.suptitle(title)
    
    
    
    
    # forward steps
    a = fig.add_subplot(spec[0, 0])
    
    
    xlabel = list(range(1, forward_k+1))
    plt.xticks(xlabel)
    a.set_ylim(0.0, 0.5)
    a.set_title('Prediction Error vs Forward Steps, theta_dim = {}'.format(k+theta_dim), color='C0')
    for i in iterations:
        show = error_for_all_theta_dim[k][i] 
        a.plot([i+1 for i in range(forward_k)], show, marker='^', markersize= 5, linewidth=1)
    a.legend(["em itr={}".format(i) for i in iterations])
    
    b = fig.add_subplot(spec[0, 1])
    b.xaxis_date()
    # volatility trajectory
    
    
    averager = np.ones(2*k+8) / (2*k+8)
    b.set_title('Volatility Trajectory vs theta dimension, theta_dim = {}'.format(k+8), color='C0')
    
    legend = []
    
    opaque = 0.9/(len(iterations)+1)
    for i in iterations:
        smoothed_volatility_mean = np.average(ms_s_for_all_theta_dim[k][i+1], axis=1)
        smoothed_volatility_std = np.array([math.sqrt(np.matmul(np.matmul(averager.T, _), averager.T)) for _ in Cs_s_for_all_theta_dim[k][i+1]])
        smoothed_volatility_ci_upper = smoothed_volatility_mean + 2*smoothed_volatility_std
        smoothed_volatility_ci_lower = smoothed_volatility_mean - 2*smoothed_volatility_std
        train_predicted_volatility_mean = train_pred_ms_for_all_theta_dim[k][i]
        
        predicted_volatility_mean = np.average(pred_ms_for_all_theta_dim[k][i], axis=1)
        
        
        b.plot(all_days, smoothed_volatility_mean[1:], linewidth=1, label="em {} rts_mean".format(i), )
#         b.fill_between(all_days, smoothed_volatility_ci_upper[1:], smoothed_volatility_ci_lower[1:], alpha = 0.3*opaque*(1+i-mean_iteration), label="em {} CI".format(i))
        b.plot(all_days, train_predicted_volatility_mean, linewidth=1, color = "green", label="em {} train_pred".format(i),)
        b.plot(all_pred_days[:-1], predicted_volatility_mean, linewidth=1, color = "red", label="em {} pred".format(i))
    
#         legend.append("em {} rts_mean".format(i))
#         legend.append("em {} train_pred".format(i))
#         legend.append("em {} pred".format(i))
#         legend.append("em {} CI".format(i))
        
#         print(np.average(np.abs(params_for_all_theta_dim[k][i][0])))
#         print(np.average(np.abs(params_for_all_theta_dim[k][i][1])))
#         print(np.average(np.abs(params_for_all_theta_dim[k][i][2])))
#         print(np.average(np.abs(params_for_all_theta_dim[k][i][3])))
#         print()
    
    b.legend()
    
    plt.xticks(rotation=60)
#     plt.savefig("./output_fig/Final.jpg")
    plt.show()
        
print(show[0])
print(show[4])
print(show[9])
print(show[14])
print(show[19])

FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000002E4957323A8> (for post_execute):


ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 1656x432 with 2 Axes>

In [148]:
print(len(error_for_all_theta_dim))
print(len(best_itr_for_all_theta_dim))
print(len(best_train_itr_for_all_theta_dim))
print(len(params_for_all_theta_dim))
print(len(ms_s_for_all_theta_dim))
print(len(Cs_s_for_all_theta_dim))
print(len(pred_ms_for_all_theta_dim))
print(len(train_pred_ms_for_all_theta_dim))

[[[0.5115099384453947, 0.5099788006803677, 0.5007509117835236, 0.4782081432021517, 0.4707445527472029, 0.45793206064855885, 0.45610123522432694, 0.4536740879016493, 0.4474739986212312, 0.42442854706063027, 0.4144274789172402, 0.40156986983797976, 0.40444448539778544, 0.40670054143050377, 0.41900179687454797, 0.4065098662706654, 0.4080431145248659, 0.4039725798767379, 0.443038213702192, 0.46404075245935844], [0.48921765758041663, 0.48850507022933987, 0.4794234345164076, 0.45663344114744864, 0.44878211458696193, 0.4354949376883835, 0.43320178516364805, 0.43015351607322516, 0.42372967371570347, 0.40002265795360425, 0.3896209098628425, 0.37690486513367105, 0.3801460923114189, 0.3816579536284853, 0.3940487288288031, 0.3800261379227588, 0.3838376844363978, 0.3785821711574643, 0.4147086949819403, 0.4343223357543664], [0.4706697709775059, 0.4709106551674137, 0.46203961004343175, 0.4391472801308136, 0.43102759462236306, 0.41739056192947177, 0.4147368228740629, 0.4111899700709791, 0.404619944742

array([-2.14843205, -2.51770665, -2.582884  , -1.4637424 , -3.60301277,
       -1.5062894 , -2.379324  , -2.19901629])